In [1]:
!pip install arxiv

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/81.1 kB ? eta -:--:--
   ---------------------------------------- 81.1/81.1 kB 2.3 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6061 sha256=eaf8804ab7c3de1c9690c776b48a3db799013d362913c87a1f37cd46dea4baa0
  Stored in directory: c:\users\kanis\appdata\local\pip\cache\wheels\f0\69\93\a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [2]:
wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=wrapper)

In [3]:
wiki.name

'wikipedia'

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS,lancedb
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader=WebBaseLoader(web_path='https://docs.smith.langchain.com/')
docs=loader.load()
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=150)
docs=splitter.split_documents(docs)
db=FAISS.from_documents(docs,OllamaEmbeddings())
retriever=db.as_retriever()

In [5]:
from langchain.tools.retriever import create_retriever_tool
ret_tool=create_retriever_tool(retriever,"langsmith_search",
                      "search for informatuon aboput langsmith.For any queries about langsmith use this tool.")


In [6]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [7]:
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [8]:
arxiv_tool.name

'arxiv'

In [9]:
tool_list=[wiki,arxiv_tool,ret_tool]

In [10]:
from langchain_community.llms import Ollama
llm=Ollama()

In [11]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [12]:
from langchain.agents import LLMSingleActionAgent,create_openai_tools_agent
agent=create_openai_tools_agent(llm,tool_list,prompt)
# LLMSingleActionAgent(llm,)

In [13]:
from langchain.agents import AgentExecutor
executor=AgentExecutor(agent=agent,tools=tool_list,verbose=True,)

In [15]:
executor.invoke({'input':'what is the encoder '})



> Entering new AgentExecutor chain...


ValueError: Ollama call failed with status code 400. Details: {"error":"invalid options: tools"}